## Clone the repository by running the cell below

In [ ]:
# --- Setup: clone lab repo and enter it ---
!git clone https://github.com/dontolon/ml_beam_management_lab.git
%cd ml_beam_management_lab


In [ ]:
%cd /home/dante/experiments/lab


### Install dependencies

In [ ]:
!pip install numpy>=1.22 pandas>=1.5 matplotlib>=3.6 torch>=2.0 scikit-learn>=1.2 utm>=0.7

### Import dependencies

In [ ]:
import numpy as np
from pathlib import Path
from helper_scripts.load_data import normalize_pos, min_max, build_mlp
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn


#### Load the data

We will import and load scenario 2 from the DeepSense6G dataset (https://www.deepsense6g.net/scenario-2/) 

This scenario emulates a Vehicle-to-Infrastructure (V2I) mmWave communication setup. 

We consider a system where a base station (BS) with $N$ antennas communicates with a single-antenna UE using one of the $M$ beamforming vectors $\mathbf{f}_m \in \mathbb{C}^{N \times 1}$ present in its codebook 
$\mathcal{F} = \{\mathbf{f}_m\}_{m=1}^M$.

In our case:  
- $M = 64$ beamforming vectors,  
- $N = 16$ antennas at the BS,  
- carrier frequency is **60 GHz**.  




In [ ]:
DATA_DIR = Path("data")

# Base station coordinates
BS_location = np.array([[33.42034722222222, -111.92915277777779]])

# UE Positions (lat, lon)
UE_locations = np.load(DATA_DIR / "scenario2_unit2_loc_1-2974.npy")[:, :2]

# Beam powers (linear scale)
beam_powers = np.load(DATA_DIR / "scenario2_unit1_pwr_60ghz_1-2974.npy")

print("BS_location:", BS_location.shape)
print("UE_locations:", UE_locations.shape)
print("beam_powers:", beam_powers.shape)

#### Preparing the data: BS and UE positions

In this step, we prepare the spatial data before visualization and modeling:

- We **stack** the base station (BS) and user equipment (UE) positions together into one array.  
- We then apply **min–max normalization** across all coordinates so that the BS and UE positions are represented in a comparable, normalized coordinate system between 0 and 1.  

This ensures that:
- Both BS and UE positions are expressed on the same scale.  
- Visualization and downstream models (e.g., beam selection models) are not biased by raw latitude/longitude magnitudes.  

After normalization:
- The **first row** corresponds to the BS.  
- The **remaining rows** correspond to the UEs.


In [ ]:
# Stack BS and UEs together
all_positions = np.vstack([BS_location, UE_locations])  # (1+N, 2)

# Min-max normalize everything
all_norm = min_max(all_positions, axis=0)

# First row is BS, rest are UEs
BS_norm = all_norm[0:1]
UE_norm = all_norm[1:]

#### Visualizing the normalized positions

Here, we create a scatter plot to visualize the **normalized positions** of the BS and UEs:

- The BS is shown as a red star.  
- The UEs are shown as small blue dots.  

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(UE_norm[:,0], UE_norm[:,1], c="blue", s=8, label="UEs")
plt.scatter(BS_norm[0,0], BS_norm[0,1], c="red", marker="*", s=200, label="BS")
plt.xlabel("Normalized Lat")
plt.ylabel("Normalized Long")
plt.title("UE positions + BS (min–max normalized together)")
plt.legend()
plt.show()


#### Exploring beam power profiles 

Let's randomly pick one sample from the dataset and explore:

1. **Beam power profile (left plot):**  
   - Shows the received power values across all 64 beams for the chosen UE.  
   
2. **Spatial view (right plot):**  
   - Shows all normalized UE positions (blue dots).  
   - The BS is marked with a red star.  
   - The randomly selected UE is highlighted in orange. 


In [ ]:
idx = np.random.randint(0, beam_powers.shape[0])
powers = beam_powers[idx]

fig, axes = plt.subplots(1, 2, figsize=(14,5))
best_idx = np.argmax(powers)

# --- Left: beam power profile ---
axes[0].plot(range(1, len(powers)+1), powers, marker="o")
axes[0].axvline(best_idx+1, color="red", linestyle="--", label=f"Best beam {best_idx+1}")
axes[0].set_xlabel("Beam index (1–64)")
axes[0].set_ylabel("Power value")
axes[0].set_title(f"UE {idx}: Power across beams")
axes[0].legend()
axes[0].grid(True)

# --- Right: spatial scatter ---
axes[1].scatter(UE_norm[:,0], UE_norm[:,1], c="blue", s=8, label="UEs")
axes[1].scatter(BS_norm[0,0], BS_norm[0,1], c="red", marker="*", s=200, label="BS")
axes[1].scatter(UE_norm[idx,0], UE_norm[idx,1], c="orange", edgecolor="black", s=80, label="Selected UE")
axes[1].set_xlabel("Normalized x")
axes[1].set_ylabel("Normalized y")
axes[1].set_title("UE positions + BS (selected UE highlighted)")
axes[1].legend()

plt.tight_layout()
plt.show()


#### Constructing the beam prediction dataset

So far, we’ve visualized single UEs and their beam power profiles.  
Now we take the next step: create the dataset for supervised learning.

- For each sample, we find its **best beam index**:  
$$
m^* = \arg\max_{m \in \{1, \dots, M\}} \; \text{beam\_powers}_{m},
$$

- We then define:  
  - **Features (X):** the normalized UE positions (shape = (N, 2))  
  - **Labels (y):** the best beam index for each sample (shape = (N,))  

This dataset \((X, y)\) is the starting point for training models to **predict the best beam** from position information.


In [ ]:
best_beam = np.argmax(beam_powers, axis=1)
X = UE_norm      # shape (N,2)
y = best_beam    # shape (N,)

print("Dataset created:")
print("X:", X.shape, "y:", y.shape)


#### Visualizing spatial patterns of best beams

Now that we have constructed the dataset \((X, y)\), we can visualize how the **best beam index** varies across space:

- Each sample is plotted at its normalized position \((x, y)\).  
- The color encodes the **best beam index** \(m^*\).  
- The colorbar on the right shows the mapping from color → beam index.  

This plot lets us see the **spatial partitioning of beams**:  
- Neighboring samples often share the same best beam.  

In [ ]:
plt.figure(figsize=(10,6))
sc = plt.scatter(X[:,0], X[:,1], c=y, cmap="viridis", s=8)
plt.colorbar(sc, label="Best beam index")
plt.xlabel("Normalized x")
plt.ylabel("Normalized y")
plt.title("UE positions colored by best beam")
plt.show()


#### Preparing the dataset for PyTorch

Now that we have the feature matrix $X$ (UE positions) and the labels $y$ (best beam indices), we need to prepare them for training a neural network in PyTorch.

Steps:
1. **Convert to tensors:**  
   - `X_tensor`: the input features (shape: $N \times 2$).  
   - `y_tensor`: the target labels (shape: $N$, with values in $\{0, \dots, M-1\}$).

2. **Bundle into a dataset:**  
   - `TensorDataset` combines the inputs and labels into a format that PyTorch understands.

3. **Wrap with a DataLoader:**  
   - Handles batching (here: 128 samples per batch).  
   - Shuffles the data each epoch for better generalization.  
   - Makes it easy to iterate over the dataset in training loops.


In [ ]:
# Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Bundle into a dataset
dataset = TensorDataset(X_tensor, y_tensor)

# Create a DataLoader (mini-batches)
loader = DataLoader(dataset, batch_size=128, shuffle=True)

print("Dataset ready for NN training!")
print("Number of batches:", len(loader))


### Splitting the dataset into train/validation/test

To properly train and evaluate our model, we need to split the dataset into three parts:

1. **Training set (70%)**  
   - Used by the neural network to learn the mapping from UE positions → best beam indices.

2. **Validation set (15%)**  
   - Used to tune hyperparameters and prevent overfitting.  
   - The model never directly trains on this data.

3. **Test set (15%)**  
   - Held out until the very end.  
   - Used to evaluate the *true generalization performance*.

In [ ]:
# 70/30 split first
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
# Then split 30 into 15/15
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)

# Convert to PyTorch tensors
def make_loader(X, y, batch_size=32, shuffle=False):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)
    return DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=batch_size, shuffle=shuffle)

train_loader = make_loader(X_train, y_train, batch_size=32, shuffle=True)
val_loader   = make_loader(X_val,   y_val,   batch_size=32)
test_loader  = make_loader(X_test,  y_test,  batch_size=32)


#### Building the model and training components

The helper function `build_mlp(...)` constructs not only the neural network architecture, but also the essential components for training:

- **Model:** a configurable MLP with user-defined depth, width, and activation.  
- **Loss function (criterion):** categorical cross-entropy, suitable for multi-class classification.  
- **Optimizer:** Adam, with configurable learning rate.  
- **Scheduler:** a multi-step learning rate scheduler that reduces the learning rate by a factor $\gamma$ at predefined milestones.


In [ ]:
model, criterion, optimizer, scheduler = build_mlp(
    input_dim=2,
    output_dim=64,
    hidden_dim=256,
    hidden_layers=3,
    activation="ReLU",
    lr=1e-2,
    milestones=[20, 40],
    gamma=0.2,
)

print(model)

#### Training loop and evaluation protocol

We now implement the complete training and evaluation routine for the beam prediction model.

**1. Training step (`run_epoch`)**
- Iterates over one pass of the dataset (an epoch).  
- Sets the model to training or evaluation mode.  
- For each mini-batch:
  - Computes forward pass (`out = model(xb)`).  
  - Evaluates the loss using categorical cross-entropy.  
  - If training:
    - Clears gradients (`optimizer.zero_grad()`).  
    - Backpropagates (`loss.backward()`).  
    - Updates parameters (`optimizer.step()`).  
- Tracks average loss and top-1 accuracy for the epoch.

**2. Evaluation step (`evaluate`)**
- Runs the model in inference mode (no gradient computation).  
- Computes **top-k accuracies** for several values of \(k\) (1, 3, 5, 10):  
  - For each sample, checks whether the ground-truth beam index appears among the model’s top-\(k\) predicted beams.  
- Returns a dictionary mapping \(k \mapsto \text{accuracy}\).

**3. Training loop**
- Repeats for a fixed number of epochs (here, 60).  
- Each epoch:
  - Calls `run_epoch` on the training set.  
  - Evaluates on the validation set using `evaluate`.  
  - Updates the learning rate via the scheduler.  
- Records the training history (loss, accuracy) and validation metrics.  
- Periodically prints progress (every 10 epochs, and during the first 5).

In [ ]:
def run_epoch(loader, train=True):
    losses, correct, total = [], 0, 0
    if train: model.train()
    else: model.eval()

    for xb, yb in loader:
        if train: optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        if train:
            loss.backward()
            optimizer.step()
        losses.append(loss.item())
        pred = out.argmax(dim=1)
        correct += (pred == yb).sum().item()
        total += len(yb)
    return np.mean(losses), correct/total

EPOCHS = 60
train_hist, val_hist = [], []

def evaluate(loader, model, k_values=[1,3,5,10]):
    """
    Evaluate top-k accuracies.
    Returns dict {k: acc}.
    """
    model.eval()
    correct_topk = {k: 0 for k in k_values}
    total = 0

    with torch.no_grad():
        for xb, yb in loader:
            out = model(xb)                       # logits
            _, pred_topk = out.topk(max(k_values), dim=1)  # top-k predictions
            total += yb.size(0)
            for k in k_values:
                correct_topk[k] += (pred_topk[:, :k] == yb.unsqueeze(1)).any(dim=1).sum().item()

    return {k: correct_topk[k]/total for k in k_values}


for epoch in range(EPOCHS):
    tr_loss, tr_acc = run_epoch(train_loader, train=True)
    val_metrics = evaluate(val_loader, model, k_values=[1,3,5,10])
    scheduler.step()

    train_hist.append((tr_loss, tr_acc))
    val_hist.append(val_metrics)

    if (epoch+1) % 10 == 0 or epoch < 5:
        print(f"Epoch {epoch+1:02d}: Train acc {tr_acc:.3f}, "
              f"Val top-1 {val_metrics[1]:.3f}, top-3 {val_metrics[3]:.3f}, "
              f"top-5 {val_metrics[5]:.3f}, top-10 {val_metrics[10]:.3f}")



In [ ]:
plt.figure(figsize=(8,5))
plt.plot([v[1] for v in val_hist], label="Top-1")
plt.xlabel("Epoch")
plt.ylabel("Validation accuracy")
plt.title("Validation Top-k Accuracies")
plt.legend()
plt.show()
